In [1]:
import wandb
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import tqdm

import torch
from torch.utils.data import Dataset, DataLoader
import seaborn as sns

In [2]:
pathways_df = pd.read_csv('data/pathways.csv')
pathways_df.set_index("Object ID", inplace=True)
pathways_df["Reaction-List"] = pathways_df["Reaction-List"].str.split(" // ")
pathways_df["Species"] = pathways_df["Species"].str.split(" // ")
pathways_df["In-Pathway"] = pathways_df["In-Pathway"].str.split(" // ")
pathways_df["Super-Pathways"] = pathways_df["Super-Pathways"].str.split(" // ")

reactions_df = pd.read_csv('data/reactions.csv')
reactions_df["Left"] = reactions_df["Left"].str.split(" // ")
reactions_df["Right"] = reactions_df["Right"].str.split(" // ")
reactions_df["In-Pathway"] = reactions_df["In-Pathway"].str.split(" // ")
reactions_df["Substrates"] = reactions_df["Substrates"].str.split(" // ")
reactions_df["Enzymatic-Reaction"] = reactions_df["Enzymatic-Reaction"].str.split(" // ")
reactions_df.set_index("Object ID", inplace=True)

compounds_df = pd.read_csv("data/compounds.csv")
compounds_df.set_index("Object ID", inplace=True)

species = {}
for i, row in pathways_df.iterrows():
    if type(row["Species"]) == list:
        for s in row["Species"]:
            if s in species:
                species[s].append(i)
            else:
                species[s] = [i]

In [4]:
class SimpleTripletDataset(Dataset):
    def __init__(self):
        self.organism_pathways = species
        self.organisms = list(species.keys())
        self.pathways = set(pathways_df.index)
        
    def __len__(self):
        return len(self.organisms)
    
    def __getitem__(self, idx):
        org = self.organisms[idx]
        org_pathways = self.organism_pathways[org]
    
        pos_pathway = random.choice(org_pathways)
        neg_pathway = random.choice(list(self.pathways - set(org_pathways)))
        
        return org, pos_pathway, neg_pathway
        
        
dataset = SimpleTripletDataset()
loader = DataLoader(dataset, batch_size=4, shuffle=True)

for org, pos, neg in loader:
    print("Organism:", org)
    print("Positive Pathway:", pos)
    print("Negative Pathway:", neg)
    break


Organism: ('ORG-6284', 'TAX-575584', 'TAX-119829', 'TAX-1315281')
Positive Pathway: ('PWY-7719', 'PWY-3641', 'PWY-7633', 'PWY-7929')
Negative Pathway: ('PWY-7757', 'PWY-2541', 'GAMMAHEXCHLORDEG-PWY', 'ASPARTATE-DEG1-PWY')


In [5]:
def make_pathway_graph(
        pathway_id: str,
):
    try:
        pathway = pathways_df.loc[pathway_id]
    except KeyError:
        return None, [], []
    
    try:
        reactions = reactions_df.loc[pathway["Reaction-List"]]
    except KeyError:
        return None, [], []
    #reactions = reactions_df.loc[pathway["Reaction-List"]]
        
    edges, compound_v, reaction_v = set(), set(), set()
    for i, reaction in reactions.iterrows():
        reaction_v.add(i)
        for l in reaction["Left"]:
            compound_v.add(l)
            edges.add((l, i))
        for r in reaction["Right"]:
            compound_v.add(r)
            edges.add((i, r))
    
    vertices = list(compound_v) + list(reaction_v)
    adj_matrix = np.zeros((len(vertices), len(vertices)))
    for edge in edges:
        adj_matrix[vertices.index(edge[0]), vertices.index(edge[1])] = 1
        
    return adj_matrix, list(reaction_v), list(compound_v)

substrates_count = {}

for substrates in reactions_df["Substrates"]:
    for substrate in substrates:
        if substrate in substrates_count:
            substrates_count[substrate] += 1
        else:
            substrates_count[substrate] = 1
            
compound_popularity = pd.Series(substrates_count, name="Count").sort_values(ascending=False)

def remove_compounds_from_graph(
        adj_matrix, reactions, compounds, to_remove
):
    all_labels = compounds + reactions
    indices_to_keep = [i for (i, x) in enumerate(all_labels) if x not in to_remove]
    
    adj_matrix = adj_matrix[indices_to_keep, :][:, indices_to_keep]
    reactions = [r for r in reactions if r not in to_remove]
    compounds = [c for c in compounds if c not in to_remove]
    
    return adj_matrix, reactions, compounds

from skfp.fingerprints import AtomPairFingerprint

# TOTAL BASELINE 
def embed_graph(compounds):
    compounds = [compounds_df.loc[c, "SMILES"] for c in compounds if c in compounds_df.index]
    
    try:
        fp = AtomPairFingerprint(n_jobs=-1)
        result = fp.transform(compounds).sum(axis=0)
    except Exception as e:
        print(e)
        return np.zeros(2048)
    
    return result

In [6]:
cached_base_embeddings = {}

for pwy in tqdm.tqdm(pathways_df.index):
    graph = make_pathway_graph(pwy)
    if graph is None:
        continue
    adj_matrix, reactions, compounds = graph
    embedding = embed_graph(compounds)
    cached_base_embeddings[pwy] = torch.tensor(embedding.astype(np.float32))

  0%|▏                                                                                                                        | 4/3538 [00:07<1:36:35,  1.64s/it][23:21:48] WARNING: not removing hydrogen atom without neighbors
[23:21:48] WARNING: not removing hydrogen atom without neighbors
[23:21:48] WARNING: not removing hydrogen atom without neighbors
[23:21:48] WARNING: not removing hydrogen atom without neighbors
  0%|▎                                                                                                                          | 8/3538 [00:07<30:55,  1.90it/s][23:21:48] WARNING: not removing hydrogen atom without neighbors
[23:21:48] WARNING: not removing hydrogen atom without neighbors
  0%|▍                                                                                                                         | 13/3538 [00:07<14:28,  4.06it/s][23:21:48] WARNING: not removing hydrogen atom without neighbors
[23:21:48] WARNING: not removing hydrogen atom without neighbo

need at least one array to concatenate


[23:21:49] WARNING: not removing hydrogen atom without neighbors
  1%|█▌                                                                                                                        | 47/3538 [00:08<02:30, 23.25it/s][23:21:49] WARNING: not removing hydrogen atom without neighbors
[23:21:49] WARNING: not removing hydrogen atom without neighbors
[23:21:49] WARNING: not removing hydrogen atom without neighbors
[23:21:49] WARNING: not removing hydrogen atom without neighbors
[23:21:49] WARNING: not removing hydrogen atom without neighbors
  1%|█▊                                                                                                                        | 52/3538 [00:08<02:07, 27.29it/s][23:21:49] WARNING: not removing hydrogen atom without neighbors
[23:21:49] WARNING: not removing hydrogen atom without neighbors
[23:21:49] WARNING: not removing hydrogen atom without neighbors
[23:21:49] WARNING: not removing hydrogen atom without neighbors
[23:21:49] WARNING: not remo

need at least one array to concatenate


[23:21:50] WARNING: not removing hydrogen atom without neighbors
[23:21:50] WARNING: not removing hydrogen atom without neighbors
  3%|███▏                                                                                                                      | 92/3538 [00:09<01:23, 41.12it/s][23:21:50] WARNING: not removing hydrogen atom without neighbors
[23:21:50] WARNING: not removing hydrogen atom without neighbors
[23:21:50] WARNING: not removing hydrogen atom without neighbors
[23:21:50] WARNING: not removing hydrogen atom without neighbors
  3%|███▎                                                                                                                      | 97/3538 [00:09<01:22, 41.61it/s][23:21:50] WARNING: not removing hydrogen atom without neighbors
[23:21:50] WARNING: not removing hydrogen atom without neighbors
[23:21:50] WARNING: not removing hydrogen atom without neighbors
[23:21:50] Conflicting single bond directions around double bond at index 12.
[23:21:50]   Bo

...

[23:32:57] WARNING: not removing hydrogen atom without neighbors
 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 3507/3538 [11:19<00:28,  1.09it/s][23:33:00] WARNING: not removing hydrogen atom without neighbors
[23:33:00] WARNING: not removing hydrogen atom without neighbors
[23:33:00] WARNING: not removing hydrogen atom without neighbors
[23:33:00] WARNING: not removing hydrogen atom without neighbors
[23:33:00] WARNING: not removing hydrogen atom without neighbors
 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏| 3515/3538 [11:19<00:11,  2.07it/s][23:33:00] WARNING: not removing hydrogen atom without neighbors
[23:33:00] WARNING: not removing hydrogen atom without neighbors
[23:33:00] WARNING: not removing hydrogen atom without neighbors
[23:33:00] WARNING: not removing hydrogen atom without neighbors
[23:33:00] WARNING: not remo

need at least one array to concatenate
need at least one array to concatenate


[23:33:00] WARNING: not removing hydrogen atom without neighbors
[23:33:01] WARNING: not removing hydrogen atom without neighbors
[23:33:01] WARNING: not removing hydrogen atom without neighbors
[23:33:01] WARNING: not removing hydrogen atom without neighbors
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 3529/3538 [11:19<00:01,  4.86it/s][23:33:01] WARNING: not removing hydrogen atom without neighbors
[23:33:01] WARNING: not removing hydrogen atom without neighbors
[23:33:01] WARNING: not removing hydrogen atom without neighbors
[23:33:01] WARNING: not removing hydrogen atom without neighbors
[23:33:01] WARNING: not removing hydrogen atom without neighbors
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 3534/3538 [11:19<00:00,  6.37it/s][23:33:01] WARNING: not removing hydrogen atom without neighbors
[23:33:01] SMILES Parse Erro

Python argument types in
    FingeprintGenerator64.GetFingerprintAsNumPy(FingeprintGenerator64, NoneType)
did not match C++ signature:
    GetFingerprintAsNumPy(RDKit::FingerprintGenerator<unsigned long> const* self, RDKit::ROMol mol, boost::python::api::object fromAtoms=[], boost::python::api::object ignoreAtoms=[], int confId=-1, boost::python::api::object customAtomInvariants=[], boost::python::api::object customBondInvariants=[], boost::python::api::object additionalOutput=None)


[23:33:05] WARNING: not removing hydrogen atom without neighbors
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3538/3538 [11:26<00:00,  5.15it/s]


In [8]:
class SimplePathwayEncoder(torch.nn.Module):
    def __init__(self, embedding_dim=128):
        super(SimplePathwayEncoder, self).__init__()
        self.fc1 = torch.nn.Linear(2048, embedding_dim)
        #self.fc2 = torch.nn.Linear(512, embedding_dim)
        #self.act = torch.nn.SiLU()
            
    def forward(self, pathways):
        embeddings = []
        for p in pathways:
            if p in cached_base_embeddings:
                embeddings.append(cached_base_embeddings[p])
                continue
            graph = make_pathway_graph(p)
            adj_matrix, reactions, compounds = graph
            embedding = embed_graph(compounds)
            embeddings.append(torch.tensor(embedding.astype(np.float32)))
        x = torch.stack(embeddings)
        x = self.fc1(x)
        #x = self.act(x)
        #x = self.fc2(x)
        return x

In [23]:
def save_organism_embeddings_to_csv(embedding, species_list, file_path):
    embeddings_array = embedding.weight.detach().numpy()
    embeddings_df = pd.DataFrame(embeddings_array, index=species_list).T
    embeddings_df.to_csv(file_path, index=False)

In [ ]:
wandb.init(project='Metacyc', config={
    "learning_rate": 0.001,
    "epochs": 20000,
    "batch_size": 32,
    "embedding_dim": 256
})

config = wandb.config

dataset = SimpleTripletDataset()
dataloader = DataLoader(dataset, batch_size=config.batch_size, shuffle=True)
species_list = list(species.keys())

organism_embedding = torch.nn.Embedding(len(species_list), config.embedding_dim)
pathway_encoder = SimplePathwayEncoder(config.embedding_dim)
optimizer = torch.optim.Adam(
    list(organism_embedding.parameters()) + list(pathway_encoder.parameters()), 
    lr=config.learning_rate
)

criterion = torch.nn.TripletMarginLoss(margin=1.0, p=2)


for epoch in tqdm.tqdm(range(config.epochs)):
    total_loss = 0
    num_batches = 0
    #print(f"Epoch {epoch+1}")

    for organisms, pos_pathways, neg_pathways in dataloader:
        optimizer.zero_grad()

        organism_idx = torch.tensor([species_list.index(o) for o in organisms])
        org_vectors = organism_embedding(organism_idx)
        
        pos_path_vectors = pathway_encoder(pos_pathways)
        neg_path_vectors = pathway_encoder(neg_pathways)
        
        loss = criterion(org_vectors, pos_path_vectors, neg_path_vectors)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        num_batches += 1

        # Optionally log batch-level metrics
        #wandb.log({"Batch Loss": loss.item()})

    average_loss = total_loss / num_batches
    #print(f"Average Loss for Epoch {epoch+1}: {average_loss:.4f}")

    if epoch % 1000 == 0:
        save_organism_embeddings_to_csv(
            organism_embedding,
            species_list,
            f"contrastive_embeddings_{epoch}.csv"
        )
        # Log epoch-level metrics
    wandb.log({"Epoch": epoch + 1, "Average Loss": average_loss})

wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Average Loss,█▇▄▂▂▂▃▁
Epoch,▁▂▃▄▅▆▇█
Average Loss,2.43395
Epoch,8


 11%|████████████▊                                                                                                        | 2199/20000 [58:35<7:48:57,  1.58s/it]